In [ ]:
import numpy as np
import pandas as pd
import shelve
from scipy.stats import gumbel_r, norm

# *Forward simulation* algorithm build-up and old snippets of code

In [ ]:
# CPB id creation (0-908)

# DataFrame with movie id, with 0-355 for Brazilian ones (Ancine id starting with 'B') and 356-908 for foreign ones
# (Ancine id starting with 'E')

cpb_id = painel.groupby(['CPB_ROE']).agg({'id_NAC':'first'})
cpb_id['id'] = np.arange(cpb_id.shape[0])

import shelve

with shelve.open(r'bbl.out') as ws:
    ws['cpb_id'] = cpb_id

In [ ]:
# Former data cleaning and import 

# starting with cpb_id import

filename = r'bbl.out'

with shelve.open(filename) as ws:
    cpb_id = ws['cpb_id']

painel = pd.read_csv('Painel 2018.csv', dtype={
    'ANO_CINEMATOGRAFICO':int , 'SEMANA_CINEMATOGRAFICA':int, 'REGISTRO_COMPLEXO':int,
    'CPB_ROE':str, 'OCUPAÇÃO_SALA_INFERIDA':float, 'd_t':int, 'x_t':float,
    'id_NAC':int, 'xt_comp':float, 't_comp':int, 'OBG_FINAL_COMP':float,
    'SALAS_COMP':float, 'DIA_abs':int, 'COMP_CUMPRIU':bool, 'ASSENTOS_INFERIDO':int, 'TIPO_SESSAO':str,
    'xt_frac':float}, usecols=['ANO_CINEMATOGRAFICO', 'SEMANA_CINEMATOGRAFICA',
        'REGISTRO_COMPLEXO', 'CPB_ROE', 'OCUPAÇÃO_SALA_INFERIDA', 'd_t','id_NAC', 'xt_comp', 't_comp', 
        'OBG_FINAL_COMP','SALAS_COMP','DIA_abs', 'COMP_CUMPRIU','xt_frac', 'ASSENTOS_INFERIDO', 'TIPO_SESSAO'])

# turning last cine week of 2017 --- that goes on to the beggining of calendar-year of 2018 --- to week 0, so that we
# don't have double entries in week no.: week 52 in 2016 

painel.loc[(painel.ANO_CINEMATOGRAFICO == 2017)&(painel.SEMANA_CINEMATOGRAFICA == 52), 'SEMANA_CINEMATOGRAFICA'] = 0

# removing movie theater with 0 quota
painel = painel[painel.REGISTRO_COMPLEXO != 17556]

# this col formalizes the state transition function, i.e. equals the % fulfillment in session t should the screened movie be 
    # Brazilian
painel['cump_frac'] = np.divide(np.divide(1, painel['d_t'].values), painel['OBG_FINAL_COMP'].values)

# fractional fulfilllment at session t
painel['xt_frac'] = painel['xt_comp'] / painel['OBG_FINAL_COMP']

# mapping movie ids to panel data from cpb_id array
painel['cpb_id'] = painel.loc[:,'CPB_ROE'].map(cpb_id.loc[:,'id'], na_action ='ignore')

painel.to_csv('Painel 2018 final.csv', index=False) # export

In [ ]:
np_obras = np.zeros((909,53))

# function to place in movie id (row) x week (col) the avg seat occupation if the movie was screened that week and 0 otherwise
def placer(row):
    np_obras[row['cpb_id'], row['SEMANA_CINEMATOGRAFICA']] = row['OCUPAÇÃO_SALA_INFERIDA']

obras.apply(placer, axis=1) # applying placer to object obras previously defined

with shelve.open(filename) as ws: # storing results
    ws['np_obras'] = np_obras

In [ ]:
import time
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV

grid_dict = {} # creating dict to store regression objects

cpb_index = np.arange(np_obras.shape[0]) # ids of movies

for w in range(53):
    tsw = time.localtime(time.time())
    print(f'Starting week {w} at day {tsw.tm_mday}, {tsw.tm_hour}:{tsw.tm_min}')
    painelzim = painel.query("SEMANA_CINEMATOGRAFICA==@w")[['cpb_id','DIA_abs','xt_frac']]
    for o in cpb_index[np.where(np_obras[:,w] > 0, True, False)]:        
        # now more narrowly defined for each movie id
        ds = painelzim.query("cpb_id==@o")[['DIA_abs','xt_frac']].to_numpy()
        # computing and storing KDEs
        try:
            grid = GridSearchCV(KernelDensity(), {'bandwidth': np.linspace(0,10,20)})
            grid_dict[f'{w},{int(o)},5'] = grid.fit(ds)
            print(f'{o} in week {w} proceeded to standard 5 CV KDE') # 5-fold cross validation
        except:
            try: # if sample is too low to accomodate 5-fold CV we try 2-fold CV
                grid = GridSearchCV(KernelDensity(), {'bandwidth': np.linspace(0,10,20)}, cv=2)
                grid_dict[f'{w},{int(o)},2'] = grid.fit(ds)
                print(f'{o} in week {w} proceeded to non-standard 2 CV KDE')
            except: # for cases where even 2-fold CV is not feasible, namely, for movies that were screen only once or twice
                grid_dict[f'{w},{int(o)},0'] = KernelDensity(bandwidth=0.5).fit(ds)
                print(f'{o} in week {w} proceeded to simple KDE')

In [ ]:
with shelve.open(r'bbl.out') as ws: # storing results from above
    ws['kdes_cv'] = grid_dict

# Importing data

In [ ]:
# defining cols to import from Painel 2018 final.csv

colunas = ['ANO_CINEMATOGRAFICO', 'SEMANA_CINEMATOGRAFICA', 'TIPO_SESSAO',
       'REGISTRO_COMPLEXO', 'CPB_ROE', 'ASSENTOS_INFERIDO',
       'OCUPAÇÃO_SALA_INFERIDA', 'd_t', 'id_NAC', 'xt_comp', 't_comp',
       'OBG_FINAL_COMP', 'SALAS_COMP', 'DIA_abs', 'COMP_CUMPRIU', 'xt_frac',
       'cump_frac', 'cpb_id', 'beta']

remover = {'CPB_ROE','ASSENTOS_INFERIDO','TIPO_SESSAO','ANO_CINEMATOGRAFICO','d_t'} # cols not to be features

importar = list(set(colunas).difference(remover))

In [ ]:
painel = pd.read_csv('Painel 2018 final.csv', dtype={
    'ANO_CINEMATOGRAFICO':int , 'SEMANA_CINEMATOGRAFICA':int, 'REGISTRO_COMPLEXO':int,
    'CPB_ROE':str, 'OCUPAÇÃO_SALA_INFERIDA':float, 'd_t':int, 'x_t':float,
    'id_NAC':int, 'xt_comp':float, 't_comp':int, 'OBG_FINAL_COMP':float,
    'SALAS_COMP':float, 'DIA_abs':int, 'COMP_CUMPRIU':bool, 'cpb_id':int, 'cump_frac':float, 
    'xt_frac':float, 'ASSENTOS_INFERIDO':int, 'TIPO_SESSAO':str, 'beta':float}, usecols=importar)

In [ ]:
# dataframe with number of sessions (t) or observations for each movie theater. this value goes as an arg for simulations
compobs = painel.groupby(['REGISTRO_COMPLEXO']).t_comp.count()

#  Simulations

## Importing stored variables and getting densities

In [ ]:
with shelve.open(r'bbl.out') as ws:
    np_obras = ws['np_obras'] # defined in the old snippets of code above, 908 rows vs. 53 cols, movie ids and weeks resp.
    kdes_cv = ws['kdes_cv'] # KDEs obtained via GridSearchCV
    logits = ws['logits_regs_all'] # first-stage Logit CCPs
    cols = ws['logits_cols_all'] # cols from Logits

In [ ]:
import bbl # py script with functions for sims

In [ ]:
# getting density by traditional KDE w/o GridSearchCV. in this case, bandwidth was computed using the mode of BWs of a
# sample where GridSearch was applied

d = bbl.get_kdes(painel,np_obras)

In [ ]:
# alternative dict getting items from kdes with bandwidth computed with GridSearchCV

d = {}

for k, v in kdes_cv.items():
    w, cpb, cv = k.split(',')
    if cv == '0': # these were already directly calculated because sample didn't allow for N-fold CV
        d[f'{w},{int(cpb)}'] = v
    else: # from these we get GridSearchCV best estimator attribute
        d[f'{w},{int(cpb)}'] = v.best_estimator_

## Defining function args and movie theater list

In [ ]:
c_list = list(painel.query("SALAS_COMP == 12").REGISTRO_COMPLEXO.unique()) # list of movie theater complexes of 12 screens
obs = [compobs.loc[c] for c in c_list] # observations or sessions from DataFrame (see sec. 2)

# choosing only relevant panel variables for each movie theater, for details see, e.g., cval in bbl.py
pc_np_899 = painel[painel.REGISTRO_COMPLEXO == 899][['cump_frac','DIA_abs','SEMANA_CINEMATOGRAFICA']].to_numpy()
pc_np_2616 = painel[painel.REGISTRO_COMPLEXO == 2616][['cump_frac','DIA_abs','SEMANA_CINEMATOGRAFICA']].to_numpy()
pc_np_6586 = painel[painel.REGISTRO_COMPLEXO == 6586][['cump_frac','DIA_abs','SEMANA_CINEMATOGRAFICA']].to_numpy()
pc_np_24763 = painel[painel.REGISTRO_COMPLEXO == 24763][['cump_frac','DIA_abs','SEMANA_CINEMATOGRAFICA']].to_numpy()
pc_np_30155 = painel[painel.REGISTRO_COMPLEXO == 30155][['cump_frac','DIA_abs','SEMANA_CINEMATOGRAFICA']].to_numpy()
pc_np_30352 = painel[painel.REGISTRO_COMPLEXO == 30352][['cump_frac','DIA_abs','SEMANA_CINEMATOGRAFICA']].to_numpy()

pc_nps = [pc_np_899, pc_np_2616, pc_np_6586, pc_np_24763, pc_np_30155, pc_np_30352]

zipped = list(zip(c_list, obs, pc_nps)) # zipping list to facilitate multiprocessing 

In [ ]:
print(painel.query("SALAS_COMP == 12").REGISTRO_COMPLEXO.unique()) # chosen movie theater ids

## Cval simulations (see bbl.py for more info)

Cval stands for correct agent value function

### KDEs

In [ ]:
# parallelizing the process with joblib

from joblib import Parallel, delayed

func = bbl.cval # for details, check bbl.py

avg = 40

cval_899, cval_2616, cval_6586, cval_24763, cval_30155, cval_30352 = Parallel(
    n_jobs = 6, backend='multiprocessing')(delayed(func)(c, avg, o, p, np_obras, d) for c, o, p in zipped)

In [ ]:
# multiplying by betas. recall that betas are daily discount factors for a annual interest rate of 6.5%

with shelve.open(r'bbl.out') as ws:
    for i in range(3):
        cval_899[:,i] = np.multiply(cval_899[:,i],ws['899_betas'])
        cval_2616[:,i] = np.multiply(cval_2616[:,i],ws['2616_betas'])
        cval_6586[:,i] = np.multiply(cval_6586[:,i],ws['6586_betas'])
        cval_24763[:,i] = np.multiply(cval_24763[:,i],ws['24763_betas'])
        cval_30155[:,i] = np.multiply(cval_30155[:,i],ws['30155_betas'])
        cval_30352[:,i] = np.multiply(cval_30352[:,i],ws['30352_betas'])

In [ ]:
# storing results with shelve

with shelve.open(r'bbl.out') as ws:
    ws['cval_899'] = cval_899
    ws['cval_2616'] = cval_2616
    ws['cval_6586'] = cval_6586
    ws['cval_24763'] = cval_24763
    ws['cval_30155'] = cval_30155 
    ws['cval_30352'] = cval_30352

### Logits

In [ ]:
cvals = {}

avg = 100

for c, o, p in zipped: # logit sims are substantially faster so that we can run them without multiprocessing
    print(c)
    cvals[f'cval_{c}'] = bbl.cval_logit(c,avg,o,p,np_obras,logits,cols)


In [ ]:
# multiplying by betas. recall that betas are daily discount factors for a annual interest rate of 6.5%

for c in c_list: # c_list defined in section 3.2
    for i in range(3):
        cvals[f'cval_{c}'][:,i] = np.multiply(cvals[f'cval_{c}'][:,i],
                                              painel.loc[(painel.REGISTRO_COMPLEXO == c), 'beta'].values)

In [ ]:
with shelve.open(r'bbl.out') as ws: # storing results
    for c in c_list:
        ws[f'logit_cval_{c}'] = cvals[f'cval_{c}']  

## Distval simulations (see bbl.py for more info)

Distval stands for distorted agent value functions. As mentioned in the README.md, we use two approaches: systematic bias and random noise.

### KDEs

#### Distval bias

##### ID: 899

In [ ]:
from joblib import Parallel, delayed # parallel processing with joblib

func = bbl.distval

c = 899 # movie theater id

wfs = [0.1, 0.2, -0.1, -0.2] # weighting factors used to distort probabilities for Brazilian movies

pc_np = painel[painel.REGISTRO_COMPLEXO == c][['cump_frac','DIA_abs','SEMANA_CINEMATOGRAFICA']].to_numpy() # here we simulate each movie theater, so we get the panel data each time

dval_899_11, dval_899_12, dval_899_09, dval_899_08 = Parallel(n_jobs = 4, backend='multiprocessing')(delayed(func)(c, 30, compobs.loc[c], pc_np, np_obras, i, d) for i in wfs)

In [ ]:
dval_899 = np.zeros(shape=(dval_899_11.shape[0], dval_899_11.shape[1], 4)) # create ndarray to store results

# and store
dval_899[:,:,0], dval_899[:,:,1], dval_899[:,:,2], dval_899[:,:,3] = dval_899_11, dval_899_12, dval_899_09, dval_899_08

In [ ]:
with shelve.open(r'bbl.out') as ws:
    ws['dval_899'] = dval_899 # store raw results
    for n in range(4):
        for i in range(3):
            dval_899[:,i,n] = np.multiply(dval_899[:,i,n],painel.loc[(painel.REGISTRO_COMPLEXO == 899), 'beta'].values)
    ws['beta_dval_899'] = dval_899 # store discounted res

##### ID: 2616

In [ ]:
from joblib import Parallel, delayed # parallel processing with joblib

func = bbl.distval

c = 2616 # movie theater id

wfs = [0.1, 0.2, -0.1, -0.2] # weighting factors used to distort probabilities for Brazilian movies

pc_np = painel[painel.REGISTRO_COMPLEXO == c][['cump_frac','DIA_abs','SEMANA_CINEMATOGRAFICA']].to_numpy() # here we simulate each movie theater, so we get the panel data each time

dval_2616_11, dval_2616_12, dval_2616_09, dval_2616_08 = Parallel(n_jobs = 4, backend='multiprocessing')(delayed(func)(c, 30, compobs.loc[c], pc_np, np_obras, i, d) for i in wfs)

In [ ]:
dval_2616 = np.zeros(shape=(dval_2616_11.shape[0], dval_2616_11.shape[1], 4)) # create ndarray to store results

# and store
dval_2616[:,:,0], dval_2616[:,:,1], dval_2616[:,:,2], dval_2616[:,:,3] = dval_2616_11, dval_2616_12, dval_2616_09, dval_2616_08

In [ ]:
with shelve.open(r'bbl.out') as ws:
    ws['dval_2616'] = dval_2616 # store raw results
    for n in range(4):
        for i in range(3):
            dval_2616[:,i,n] = np.multiply(dval_2616[:,i,n],painel.loc[(painel.REGISTRO_COMPLEXO == 2616), 'beta'].values)
    ws['beta_dval_2616'] = dval_2616 # store discounted res

##### ID: 6586

In [ ]:
from joblib import Parallel, delayed # parallel processing with joblib

func = bbl.distval

c = 6586 # movie theater id

wfs = [0.1, 0.2, -0.1, -0.2] # weighting factors used to distort probabilities for Brazilian movies

pc_np = painel[painel.REGISTRO_COMPLEXO == c][['cump_frac','DIA_abs','SEMANA_CINEMATOGRAFICA']].to_numpy() # here we simulate each movie theater, so we get the panel data each time

dval_6586_11, dval_6586_12, dval_6586_09, dval_6586_08 = Parallel(n_jobs = 4, backend='multiprocessing')(delayed(func)(c, 30, compobs.loc[c], pc_np, np_obras, i, d) for i in wfs)

In [ ]:
dval_6586 = np.zeros(shape=(dval_6586_11.shape[0], dval_6586_11.shape[1], 4)) # create ndarray to store results

# and store
dval_6586[:,:,0], dval_6586[:,:,1], dval_6586[:,:,2], dval_6586[:,:,3] = dval_6586_11, dval_6586_12, dval_6586_09, dval_6586_08

In [ ]:
with shelve.open(r'bbl.out') as ws:
    ws['dval_6586'] = dval_6586 # store raw results
    for n in range(4):
        for i in range(3):
            dval_6586[:,i,n] = np.multiply(dval_6586[:,i,n],painel.loc[(painel.REGISTRO_COMPLEXO == 6586), 'beta'].values)
    ws['beta_dval_6586'] = dval_6586 # store discounted res

##### ID: 24763

In [ ]:
from joblib import Parallel, delayed # parallel processing with joblib

func = bbl.distval

c = 24763 # movie theater id

wfs = [0.1, 0.2, -0.1, -0.2] # weighting factors used to distort probabilities for Brazilian movies

pc_np = painel[painel.REGISTRO_COMPLEXO == c][['cump_frac','DIA_abs','SEMANA_CINEMATOGRAFICA']].to_numpy() # here we simulate each movie theater, so we get the panel data each time

dval_24763_11, dval_24763_12, dval_24763_09, dval_24763_08 = Parallel(n_jobs = 4, backend='multiprocessing')(delayed(func)(c, 30, compobs.loc[c], pc_np, np_obras, i, d) for i in wfs)

In [ ]:
dval_24763 = np.zeros(shape=(dval_24763_11.shape[0], dval_24763_11.shape[1], 4)) # create ndarray to store results

# and store
dval_24763[:,:,0], dval_24763[:,:,1], dval_24763[:,:,2], dval_24763[:,:,3] = dval_24763_11, dval_24763_12, dval_24763_09, dval_24763_08

In [ ]:
with shelve.open(r'bbl.out') as ws:
    ws['dval_24763'] = dval_24763 # store raw results
    for n in range(4):
        for i in range(3):
            dval_24763[:,i,n] = np.multiply(dval_24763[:,i,n],painel.loc[(painel.REGISTRO_COMPLEXO == 24763), 'beta'].values)
    ws['beta_dval_24763'] = dval_24763 # store discounted res

##### ID: 30155

In [ ]:
from joblib import Parallel, delayed # parallel processing with joblib

func = bbl.distval

c = 30155 # movie theater id

wfs = [0.1, 0.2, -0.1, -0.2] # weighting factors used to distort probabilities for Brazilian movies

pc_np = painel[painel.REGISTRO_COMPLEXO == c][['cump_frac','DIA_abs','SEMANA_CINEMATOGRAFICA']].to_numpy() # here we simulate each movie theater, so we get the panel data each time

dval_30155_11, dval_30155_12, dval_30155_09, dval_30155_08 = Parallel(n_jobs = 4, backend='multiprocessing')(delayed(func)(c, 30, compobs.loc[c], pc_np, np_obras, i, d) for i in wfs)

In [ ]:
dval_30155 = np.zeros(shape=(dval_30155_11.shape[0], dval_30155_11.shape[1], 4)) # create ndarray to store results

# and store
dval_30155[:,:,0], dval_30155[:,:,1], dval_30155[:,:,2], dval_30155[:,:,3] = dval_30155_11, dval_30155_12, dval_30155_09, dval_30155_08

In [ ]:
with shelve.open(r'bbl.out') as ws:
    ws['dval_6586'] = dval_6586 # store raw results
    for n in range(4):
        for i in range(3):
            dval_6586[:,i,n] = np.multiply(dval_6586[:,i,n],painel.loc[(painel.REGISTRO_COMPLEXO == 6586), 'beta'].values)
    ws['beta_dval_6586'] = dval_6586 # store discounted res

##### ID: 30352

In [ ]:
from joblib import Parallel, delayed # parallel processing with joblib

func = bbl.distval

c = 30352 # movie theater id

wfs = [0.1, 0.2, -0.1, -0.2] # weighting factors used to distort probabilities for Brazilian movies

pc_np = painel[painel.REGISTRO_COMPLEXO == c][['cump_frac','DIA_abs','SEMANA_CINEMATOGRAFICA']].to_numpy() # here we simulate each movie theater, so we get the panel data each time

dval_30352_11, dval_30352_12, dval_30352_09, dval_30352_08 = Parallel(n_jobs = 4, backend='multiprocessing')(delayed(func)(c, 30, compobs.loc[c], pc_np, np_obras, i, d) for i in wfs)

In [ ]:
dval_30352 = np.zeros(shape=(dval_30352_11.shape[0], dval_30352_11.shape[1], 4)) # create ndarray to store results

# and store
dval_30352[:,:,0], dval_30352[:,:,1], dval_30352[:,:,2], dval_30352[:,:,3] = dval_30352_11, dval_30352_12, dval_30352_09, dval_30352_08

In [ ]:
with shelve.open(r'bbl.out') as ws:
    ws['dval_30352'] = dval_30352 # store raw results
    for n in range(4):
        for i in range(3):
            dval_30352[:,i,n] = np.multiply(dval_30352[:,i,n],painel.loc[(painel.REGISTRO_COMPLEXO == 30352), 'beta'].values)
    ws['beta_dval_30352'] = dval_30352 # store discounted res

#### Distval noise

In [ ]:
from joblib import Parallel, delayed

func = bbl.distval_noise # we can run these a couple of times to get more estimates

dval_899, dval_2616, dval_6586, dval_24763, dval_30155, dval_30352 = Parallel(
    n_jobs = 4, backend='multiprocessing')(delayed(func)(c, 20, o, p, np_obras, d) for c, o, p in zipped) 

In [ ]:
# multiplying by betas

for c in c_list:
    for i in range(3):
        dval_899[:,i] = np.multiply(dval_899[:,i], painel.loc[(painel.REGISTRO_COMPLEXO == 899), 'beta'].values)
        dval_2616[:,i] = np.multiply(dval_2616[:,i], painel.loc[(painel.REGISTRO_COMPLEXO == 2616), 'beta'].values)
        dval_6586[:,i] = np.multiply(dval_6586[:,i], painel.loc[(painel.REGISTRO_COMPLEXO == 6586), 'beta'].values)
        dval_24763[:,i] = np.multiply(dval_24763[:,i], painel.loc[(painel.REGISTRO_COMPLEXO == 24763), 'beta'].values)
        dval_30155[:,i] = np.multiply(dval_30155[:,i], painel.loc[(painel.REGISTRO_COMPLEXO == 30155), 'beta'].values)
        dval_30352[:,i] = np.multiply(dval_30352[:,i], painel.loc[(painel.REGISTRO_COMPLEXO == 30352), 'beta'].values)

In [ ]:
with shelve.open(r'bbl.out') as ws: # storing results
    ws['dval_899_noise'] = dval_899
    ws['dval_2616_noise'] = dval_2616
    ws['dval_6586_noise'] = dval_6586
    ws['dval_24763_noise'] = dval_24763
    ws['dval_30155_noise'] = dval_30155 
    ws['dval_30352_noise'] = dval_30352

### Logits

#### Distval bias

##### ID: 899

In [ ]:
from joblib import Parallel, delayed # parallel processing with joblib

func = bbl.distval_logit

c = 899 # movie theater id

wfs = [0.1, 0.2, -0.1, -0.2] # weighting factors

pc_np = painel[painel.REGISTRO_COMPLEXO == c][['cump_frac','DIA_abs','SEMANA_CINEMATOGRAFICA']].to_numpy()

dval_899_11, dval_899_12, dval_899_09, dval_899_08 = Parallel(n_jobs = 4, backend='multiprocessing')(delayed(func)(c, 100, compobs.loc[c], pc_np, np_obras, i, logits, cols) for i in wfs)

In [ ]:
dval_899 = np.zeros(shape=(dval_899_11.shape[0], dval_899_11.shape[1], 4)) # create array to store results

# and store
dval_899[:,:,0], dval_899[:,:,1], dval_899[:,:,2], dval_899[:,:,3] = dval_899_11, dval_899_12, dval_899_09, dval_899_08

In [ ]:
with shelve.open(r'bbl.out') as ws:
    ws['logit_dval_899'] = dval_899 # storing raw logit results
    for n in range(4):
        for i in range(3):
            dval_899[:,i,n] = np.multiply(dval_899[:,i,n],painel.loc[(painel.REGISTRO_COMPLEXO == 899), 'beta'].values)
    ws['logit_beta_dval_899'] = dval_899 # storing discounted results

##### ID: 2616

In [ ]:
from joblib import Parallel, delayed # parallel processing with joblib

func = bbl.distval_logit

c = 2616 # movie theater id

wfs = [0.1, 0.2, -0.1, -0.2] # weighting factors

pc_np = painel[painel.REGISTRO_COMPLEXO == c][['cump_frac','DIA_abs','SEMANA_CINEMATOGRAFICA']].to_numpy()

dval_2616_11, dval_2616_12, dval_2616_09, dval_2616_08 = Parallel(n_jobs = 4, backend='multiprocessing')(delayed(func)(c, 100, compobs.loc[c], pc_np, np_obras, i, logits, cols) for i in wfs)

In [ ]:
dval_2616 = np.zeros(shape=(dval_2616_11.shape[0], dval_2616_11.shape[1], 4)) # create array to store results

# and store
dval_2616[:,:,0], dval_2616[:,:,1], dval_2616[:,:,2], dval_2616[:,:,3] = dval_2616_11, dval_2616_12, dval_2616_09, dval_2616_08

In [ ]:
with shelve.open(r'bbl.out') as ws:
    ws['logit_dval_2616'] = dval_2616 # storing raw logit results
    for n in range(4):
        for i in range(3):
            dval_2616[:,i,n] = np.multiply(dval_2616[:,i,n],painel.loc[(painel.REGISTRO_COMPLEXO == 2616), 'beta'].values)
    ws['logit_beta_dval_2616'] = dval_2616 # storing discounted results

##### ID: 6586

In [ ]:
from joblib import Parallel, delayed # parallel processing with joblib

func = bbl.distval_logit

c = 6586 # movie theater id

wfs = [0.1, 0.2, -0.1, -0.2] # weighting factors

pc_np = painel[painel.REGISTRO_COMPLEXO == c][['cump_frac','DIA_abs','SEMANA_CINEMATOGRAFICA']].to_numpy()

dval_6586_11, dval_6586_12, dval_6586_09, dval_6586_08 = Parallel(n_jobs = 4, backend='multiprocessing')(delayed(func)(c, 100, compobs.loc[c], pc_np, np_obras, i, logits, cols) for i in wfs)

In [ ]:
dval_6586 = np.zeros(shape=(dval_6586_11.shape[0], dval_6586_11.shape[1], 4)) # create array to store results

# and store
dval_6586[:,:,0], dval_6586[:,:,1], dval_6586[:,:,2], dval_6586[:,:,3] = dval_6586_11, dval_6586_12, dval_6586_09, dval_6586_08

In [ ]:
with shelve.open(r'bbl.out') as ws:
    ws['logit_dval_6586'] = dval_6586 # storing raw logit results
    for n in range(4):
        for i in range(3):
            dval_6586[:,i,n] = np.multiply(dval_6586[:,i,n],painel.loc[(painel.REGISTRO_COMPLEXO == 6586), 'beta'].values)
    ws['logit_beta_dval_6586'] = dval_6586 # storing discounted results

##### ID: 24763

In [ ]:
from joblib import Parallel, delayed # parallel processing with joblib

func = bbl.distval_logit

c = 24763 # movie theater id

wfs = [0.1, 0.2, -0.1, -0.2] # weighting factors

pc_np = painel[painel.REGISTRO_COMPLEXO == c][['cump_frac','DIA_abs','SEMANA_CINEMATOGRAFICA']].to_numpy()

dval_24763_11, dval_24763_12, dval_24763_09, dval_24763_08 = Parallel(n_jobs = 4, backend='multiprocessing')(delayed(func)(c, 100, compobs.loc[c], pc_np, np_obras, i, logits, cols) for i in wfs)

In [ ]:
dval_24763 = np.zeros(shape=(dval_24763_11.shape[0], dval_24763_11.shape[1], 4)) # create array to store results

# and store
dval_24763[:,:,0], dval_24763[:,:,1], dval_24763[:,:,2], dval_24763[:,:,3] = dval_24763_11, dval_24763_12, dval_24763_09, dval_24763_08

In [ ]:
with shelve.open(r'bbl.out') as ws:
    ws['logit_dval_24763'] = dval_24763 # storing raw logit results
    for n in range(4):
        for i in range(3):
            dval_24763[:,i,n] = np.multiply(dval_24763[:,i,n],painel.loc[(painel.REGISTRO_COMPLEXO == 24763), 'beta'].values)
    ws['logit_beta_dval_24763'] = dval_24763 # storing discounted results

##### ID: 30155

In [ ]:
from joblib import Parallel, delayed # parallel processing with joblib

func = bbl.distval_logit

c = 30155 # movie theater id

wfs = [0.1, 0.2, -0.1, -0.2] # weighting factors

pc_np = painel[painel.REGISTRO_COMPLEXO == c][['cump_frac','DIA_abs','SEMANA_CINEMATOGRAFICA']].to_numpy()

dval_30155_11, dval_30155_12, dval_30155_09, dval_30155_08 = Parallel(n_jobs = 4, backend='multiprocessing')(delayed(func)(c, 100, compobs.loc[c], pc_np, np_obras, i, logits, cols) for i in wfs)

In [ ]:
dval_30155 = np.zeros(shape=(dval_30155_11.shape[0], dval_30155_11.shape[1], 4)) # create array to store results

# and store
dval_30155[:,:,0], dval_30155[:,:,1], dval_30155[:,:,2], dval_30155[:,:,3] = dval_30155_11, dval_30155_12, dval_30155_09, dval_30155_08

In [ ]:
with shelve.open(r'bbl.out') as ws:
    ws['logit_dval_30155'] = dval_30155 # storing raw logit results
    for n in range(4):
        for i in range(3):
            dval_30155[:,i,n] = np.multiply(dval_30155[:,i,n],painel.loc[(painel.REGISTRO_COMPLEXO == 30155), 'beta'].values)
    ws['logit_beta_dval_30155'] = dval_30155 # storing discounted results

##### ID: 30352

In [ ]:
from joblib import Parallel, delayed # parallel processing with joblib

func = bbl.distval_logit

c = 30352 # movie theater id

wfs = [0.1, 0.2, -0.1, -0.2] # weighting factors

pc_np = painel[painel.REGISTRO_COMPLEXO == c][['cump_frac','DIA_abs','SEMANA_CINEMATOGRAFICA']].to_numpy()

dval_30352_11, dval_30352_12, dval_30352_09, dval_30352_08 = Parallel(n_jobs = 4, backend='multiprocessing')(delayed(func)(c, 100, compobs.loc[c], pc_np, np_obras, i, logits, cols) for i in wfs)

In [ ]:
dval_30352 = np.zeros(shape=(dval_30352_11.shape[0], dval_30352_11.shape[1], 4)) # create array to store results

# and store
dval_30352[:,:,0], dval_30352[:,:,1], dval_30352[:,:,2], dval_30352[:,:,3] = dval_30352_11, dval_30352_12, dval_30352_09, dval_30352_08

In [ ]:
with shelve.open(r'bbl.out') as ws:
    ws['logit_dval_30352'] = dval_30352 # storing raw logit results
    for n in range(4):
        for i in range(3):
            dval_30352[:,i,n] = np.multiply(dval_30352[:,i,n],painel.loc[(painel.REGISTRO_COMPLEXO == 30352), 'beta'].values)
    ws['logit_beta_dval_30352'] = dval_30352 # storing discounted results

#### Distval noise

In [ ]:
from joblib import Parallel, delayed

func = bbl.distval_noise_logit

dval_899, dval_2616, dval_6586, dval_24763, dval_30155, dval_30352 = Parallel(
    n_jobs = 6, backend='multiprocessing')(delayed(func)(c, 5, o, p, np_obras, logits, cols) for c, o, p in zipped)

In [ ]:

for c in c_list: # multiplying by betas
    for i in range(3):
        dval_899[:,i] = np.multiply(dval_899[:,i], painel.loc[(painel.REGISTRO_COMPLEXO == 899), 'beta'].values)
        dval_2616[:,i] = np.multiply(dval_2616[:,i], painel.loc[(painel.REGISTRO_COMPLEXO == 2616), 'beta'].values)
        dval_6586[:,i] = np.multiply(dval_6586[:,i], painel.loc[(painel.REGISTRO_COMPLEXO == 6586), 'beta'].values)
        dval_24763[:,i] = np.multiply(dval_24763[:,i], painel.loc[(painel.REGISTRO_COMPLEXO == 24763), 'beta'].values)
        dval_30155[:,i] = np.multiply(dval_30155[:,i], painel.loc[(painel.REGISTRO_COMPLEXO == 30155), 'beta'].values)
        dval_30352[:,i] = np.multiply(dval_30352[:,i], painel.loc[(painel.REGISTRO_COMPLEXO == 30352), 'beta'].values)

In [ ]:
with shelve.open(r'bbl.out') as ws: # and storing
    ws['logit_dval_899_noise'] = dval_899
    ws['logit_dval_2616_noise'] = dval_2616
    ws['logit_dval_6586_noise'] = dval_6586
    ws['logit_dval_24763_noise'] = dval_24763
    ws['logit_dval_30155_noise'] = dval_30155 
    ws['logit_dval_30352_noise'] = dval_30352

# Compiling result arrays for each movie theater

In [ ]:
c_list = list(painel.query("SALAS_COMP == 12").REGISTRO_COMPLEXO.unique()) # same as in section 3.2

## KDE results

In [ ]:
with shelve.open(r'bbl.out') as ws:
    for c in c_list:
        cval_orig = ws[f'cval_{c}'] # getting cval for each id
        dval_orig = ws[f'beta_dval_{c}'] # getting bias distvals
        noise = ws[f'dval_{c}_noise'] # getting noisy distvals
        matrix = np.zeros((dval_orig.shape[0], dval_orig.shape[1], dval_orig.shape[2]+2)) # creating array to accomodate all
        matrix[:,:,0] = cval_orig # store orig in pos 0
        matrix[:,:,1:dval_orig.shape[2]+1] = dval_orig # dvals in between
        matrix[:,:,-1] = noise # noise for last
        ws[f'{c}_completo2'] = matrix # store complete results

## Logit results

In [ ]:
with shelve.open(r'bbl.out') as ws: 
    for c in c_list: # see 4.1 above
        cval_orig = ws[f'logit_cval_{c}']
        dval_orig = ws[f'logit_beta_dval_{c}']
        noise = ws[f'logit_dval_{c}_noise']
        matrix = np.zeros((dval_orig.shape[0], dval_orig.shape[1], dval_orig.shape[2]+2))
        matrix[:,:,0] = cval_orig
        matrix[:,:,1:dval_orig.shape[2]+1] = dval_orig
        matrix[:,:,-1] = noise
        ws[f'{c}_completo_logit'] = matrix